In [1]:
'''Import necessary packages'''
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.optimizers import Adam
import warnings
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from matplotlib import pyplot as plt
from pandas.api.types import is_string_dtype,is_numeric_dtype
import seaborn as sns
import numpy as np
import pandas as pd


Using TensorFlow backend.


In [2]:
df = pd.read_csv(f'/Users/qing/Desktop/study/data science/HW/hw3/BlackFriday.csv')
print(df.shape)
df.head(3)

(537577, 12)


,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3,Purchase
0,1000001,P00069042,F,0-17,10,A,2,0,3,NaN,NaN,8370
1,1000001,P00248942,F,0-17,10,A,2,0,1,6.0,14.0,15200
2,1000001,P00087842,F,0-17,10,A,2,0,12,NaN,NaN,1422


# Pre-process, split train-test 

In [3]:
X = df.iloc[:, :11]
Y = df.iloc[:, 11:]
print(X.head(2))

   User_ID Product_ID Gender   Age  Occupation City_Category  \
0  1000001  P00069042      F  0-17          10             A   
1  1000001  P00248942      F  0-17          10             A   

  Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
0                          2               0                   3   
1                          2               0                   1   

   Product_Category_2  Product_Category_3  
0                 NaN                 NaN  
1                 6.0                14.0  


In [4]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 11 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null object
Gender                        537577 non-null object
Age                           537577 non-null object
Occupation                    537577 non-null int64
City_Category                 537577 non-null object
Stay_In_Current_City_Years    537577 non-null object
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            370591 non-null float64
Product_Category_3            164278 non-null float64
dtypes: float64(2), int64(4), object(5)
memory usage: 45.1+ MB


In [5]:
X.fillna(value = 0, inplace = True)
X["Product_Category_2"] = X["Product_Category_2"].astype('int64')
X["Product_Category_3"] = X["Product_Category_3"].astype('int64')

In [6]:
def str_to_cat(my_df):
    for label, content in my_df.items():
        if is_string_dtype(content):
            my_df[label] = content.astype('category').cat.as_ordered()

In [7]:
str_to_cat(X)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 11 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null category
Gender                        537577 non-null category
Age                           537577 non-null category
Occupation                    537577 non-null int64
City_Category                 537577 non-null category
Stay_In_Current_City_Years    537577 non-null category
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            537577 non-null int64
Product_Category_3            537577 non-null int64
dtypes: category(5), int64(6)
memory usage: 27.9 MB


In [9]:
from fancyimpute import SoftImpute

def mydf_to_nums(my_df, feature, null_status):
    if not is_numeric_dtype(feature):
        my_df[null_status] = feature.cat.codes + 1
        my_df[null_status] = my_df[null_status].astype('int64')

def mydf_preprocessor(my_df):
    for label, content in my_df.items(): 
        mydf_to_nums(my_df, content, label)
    my_df.info()
    
mydf_preprocessor(X)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 537577 entries, 0 to 537576
Data columns (total 11 columns):
User_ID                       537577 non-null int64
Product_ID                    537577 non-null int64
Gender                        537577 non-null int64
Age                           537577 non-null int64
Occupation                    537577 non-null int64
City_Category                 537577 non-null int64
Stay_In_Current_City_Years    537577 non-null int64
Marital_Status                537577 non-null int64
Product_Category_1            537577 non-null int64
Product_Category_2            537577 non-null int64
Product_Category_3            537577 non-null int64
dtypes: int64(11)
memory usage: 45.1 MB


In [10]:
X.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
0,1000001,671,1,1,10,1,3,0,3,0,0
1,1000001,2375,1,1,10,1,3,0,1,6,14


In [11]:
(X_train_valid, X_test, Y_train_valid, Y_test) = train_test_split(X, Y, test_size=0.2, random_state=1)
(X_train, X_valid, Y_train, Y_valid) = train_test_split(X_train_valid, Y_train_valid, test_size=0.25, random_state=1)
print(X_train.shape, X_valid.shape, X_test.shape)

(322545, 11) (107516, 11) (107516, 11)


In [12]:
mean = X_train.mean(axis = 0)
X_train -= mean
std = X_train.std(axis = 0)
X_train /= std

X_valid -= mean
X_valid /= std

X_test -= mean
X_test /= std

print(X_train.head(2))

         User_ID  Product_ID    Gender       Age  Occupation  City_Category  \
122307 -1.218111   -0.286953 -1.748722  1.110580    0.601251        1.26275   
155184  1.742657   -0.831606  0.571844  0.372072   -0.165520        1.26275   

        Stay_In_Current_City_Years  Marital_Status  Product_Category_1  \
122307                   -0.666624       -0.831088            0.722295   
155184                    0.885560       -0.831088           -0.078435   

        Product_Category_2  Product_Category_3  
122307           -1.095246           -0.619078  
155184            0.192056            2.091239  


In [13]:
X_valid.head(2)

,User_ID,Product_ID,Gender,Age,Occupation,City_Category,Stay_In_Current_City_Years,Marital_Status,Product_Category_1,Product_Category_2,Product_Category_3
286426,-0.544414,0.791380,0.571844,-1.843454,-1.085646,1.26275,0.885560,-0.831088,-0.078435,1.479358,-0.619078
10739,-0.787645,-1.582749,0.571844,1.110580,-0.932292,1.26275,1.661652,-0.831088,0.722295,1.479358,-0.619078


# Build DNN

In [26]:
warnings.filterwarnings('ignore')
model = Sequential()
model.add(Dense(64, input_dim=11, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1))

In [27]:
model.compile(optimizer = 'rmsprop', loss = 'mse', metrics=['mae'])

In [28]:
history = model.fit(X_train, Y_train, validation_data=(X_valid, Y_valid), nb_epoch=100, batch_size=16)

Train on 322545 samples, validate on 107516 samples
Epoch 1/100
322545/322545 [==============================] - 16s 49us/step - loss: 25335601.9820 - mean_absolute_error: 3733.5987 - val_loss: 17809805.2272 - val_mean_absolute_error: 3138.5165
Epoch 2/100
322545/322545 [==============================] - 16s 48us/step - loss: 19257050.6659 - mean_absolute_error: 3264.9716 - val_loss: 17234088.0451 - val_mean_absolute_error: 3073.5737
Epoch 3/100
322545/322545 [==============================] - 15s 48us/step - loss: 18882994.3198 - mean_absolute_error: 3223.8837 - val_loss: 16927422.9113 - val_mean_absolute_error: 3049.9192
Epoch 4/100
322545/322545 [==============================] - 16s 48us/step - loss: 18751655.5812 - mean_absolute_error: 3210.3911 - val_loss: 16875610.9280 - val_mean_absolute_error: 3040.9498
Epoch 5/100
322545/322545 [==============================] - 15s 48us/step - loss: 18704695.4476 - mean_absolute_error: 3206.2331 - val_loss: 16905534.2170 - val_mean_absolute_

322545/322545 [==============================] - 15s 47us/step - loss: 17397838.1091 - mean_absolute_error: 3090.9754 - val_loss: 15366449.3752 - val_mean_absolute_error: 2875.1069
Epoch 44/100
322545/322545 [==============================] - 16s 48us/step - loss: 17417077.3239 - mean_absolute_error: 3090.9724 - val_loss: 15294607.2423 - val_mean_absolute_error: 2870.3943
Epoch 45/100
322545/322545 [==============================] - 15s 48us/step - loss: 17363239.0356 - mean_absolute_error: 3087.8529 - val_loss: 15194857.2383 - val_mean_absolute_error: 2866.0771
Epoch 46/100
322545/322545 [==============================] - 16s 49us/step - loss: 17295100.9316 - mean_absolute_error: 3076.8718 - val_loss: 15173497.9404 - val_mean_absolute_error: 2859.7022
Epoch 47/100
322545/322545 [==============================] - 16s 48us/step - loss: 17276851.4955 - mean_absolute_error: 3071.4795 - val_loss: 15084303.6897 - val_mean_absolute_error: 2842.6649
Epoch 48/100
322545/322545 [===============

In [29]:
test_mse_score, test_mae_score = model.evaluate(X_test, Y_test)
print(test_mae_score)

107516/107516 [==============================] - 1s 10us/step
2790.192095282621
